In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import gc
import time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from time import gmtime, strftime

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_squared_log_error
import lightgbm as lgb


import scipy
import gc
import sys

#Add https://www.kaggle.com/anttip/wordbatch to your kernel Data Sources, 
#until Kaggle admins fix the wordbatch pip package installation

sys.path.insert(0, '../input/wordbatch/wordbatch/')
import wordbatch
from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL, FM_FTRL

from nltk.corpus import stopwords
import re

def rmsle(y, y_pred):
     assert len(y) == len(y_pred)
     y = np.array(y)
     y_pred = np.array(y_pred)
     y_pred[y_pred<0] = 0
     y[y<0] = 0   
     
     return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_pred), 2)))

def rmsle2(y_true, y_pred):
    y_pred[y_pred<0]= 0
    return  np.sqrt(np.square(np.log(y_true + 1) - np.log(y_pred + 1)).mean())

In [2]:
NUM_BRANDS = 4500
NUM_CATEGORIES = 1250

develop = False
# develop= True

start_time = time.time()
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2018-02-04 18:48:02


In [3]:
train.sample??

Object `train.sample` not found.


In [4]:
LOAD_TEST = True
MAKE_SAMPLE = False

train = pd.read_table('input/train.tsv', engine='c')
if MAKE_SAMPLE:
    train = pd.read_table('input/train.tsv', engine='c')
    train = train.sample(n=50000).reset_index().drop('index',axis=1).copy(deep=True)
else:
    train = pd.read_table('input/train.tsv', engine='c')

print('Train shape: ', train.shape)
merged = pd.concat([train])
nrow_train = train.shape[0]
del train


if LOAD_TEST:
    if MAKE_SAMPLE:
        test = pd.read_table('input/test.tsv', engine='c')
        test = test.sample(n=50000).reset_index().drop('index',axis=1).copy(deep=True)
    else:
        test = pd.read_table('input/test.tsv', engine='c')

    print('Test shape: ', test.shape)
    merged = pd.concat([merged, test])
    nrow_test = test.shape[0]
    del test
    
print('Merged shape: ', merged.shape)

merged.head()

Train shape:  (1482535, 8)
Test shape:  (693359, 7)
Merged shape:  (2175894, 9)


,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id
0,NaN,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,NaN,0.0
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,NaN,1.0
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,NaN,2.0
3,NaN,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,NaN,3.0
4,NaN,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,NaN,4.0


In [5]:
## preprocess - splitting category names
##

tt = pd.DataFrame([ str(x).split("/") for x in merged['category_name']])
cat_col_names = ['general_cat','subcat_1','subcat_2','subcat_3','subcat_4']
tt.columns = ['general_cat','subcat_1','subcat_2','subcat_3','subcat_4']

merged[cat_col_names] = tt
merged["category_name"] = merged["category_name"].fillna("Other").astype("category")

## merged.drop('category_name', axis=1, inplace=True)
del tt


In [6]:
## replace missing values
for col_name in  cat_col_names+['brand_name']+['item_description']:
    merged['mis_'+col_name] = 0
    merged[col_name].fillna(value='missing', inplace=True)
    merged.loc[merged[col_name]=='missing','mis_'+col_name] = 1

merged.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,general_cat,...,subcat_2,subcat_3,subcat_4,mis_general_cat,mis_subcat_1,mis_subcat_2,mis_subcat_3,mis_subcat_4,mis_brand_name,mis_item_description
0,missing,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,NaN,0.0,Men,...,T-shirts,missing,missing,0,0,0,1,1,1,0
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,NaN,1.0,Electronics,...,Components & Parts,missing,missing,0,0,0,1,1,0,0
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,NaN,2.0,Women,...,Blouse,missing,missing,0,0,0,1,1,0,0
3,missing,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,NaN,3.0,Home,...,Home Décor Accents,missing,missing,0,0,0,1,1,1,0
4,missing,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,NaN,4.0,Women,...,Necklaces,missing,missing,0,0,0,1,1,1,0


In [7]:
VAL_MISS_MARGIN = 10

### delete rare brands
##

pop_brand = merged['brand_name'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
merged.loc[merged['brand_name'].isin(pop_brand), 'mis_brand_name'] = 1
merged.loc[merged['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'

## delete rare categories
##

pop_category1 = merged['general_cat'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
pop_category2 = merged['subcat_1'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
pop_category3 = merged['subcat_2'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
# merged.loc[merged['general_cat'].isin(pop_brand), 'mis_general_cat'] = 1
# merged.loc[merged['general_cat'].isin(pop_category1), 'general_cat'] = 'missing'
# merged.loc[merged['subcat_1'].isin(pop_brand), 'mis_subcat_1'] = 1
# merged.loc[merged['subcat_1'].isin(pop_category2), 'subcat_1'] = 'missing'
# merged.loc[merged['subcat_2'].isin(pop_brand), 'mis_subcat_2'] = 1
# merged.loc[merged['subcat_2'].isin(pop_category3), 'subcat_2'] = 'missing'
    
print (pop_brand)
print (len(pop_category1),pop_category1[1:5])
print (len(pop_category2),pop_category2[1:5])
print (len(pop_category3),pop_category3[1:5])

['Mopas' 'Coobie' 'Carole Little' ..., 'COOLMAX' 'Roses by Reem Acra'
 'Trudeau']
0 []
0 []
154 ['Telephone' 'Toys' 'Presentation' 'Kitchen Safety']


In [8]:
## https://www.kaggle.com/anttip/wordbatch-ftrl-fm-lgb-lbl-0-42555/code
## https://www.kaggle.com/anttip/wordbatch-ftrl-fm-lgb-lbl-0-42555/code
merged.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,general_cat,...,subcat_2,subcat_3,subcat_4,mis_general_cat,mis_subcat_1,mis_subcat_2,mis_subcat_3,mis_subcat_4,mis_brand_name,mis_item_description
0,missing,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,NaN,0.0,Men,...,T-shirts,missing,missing,0,0,0,1,1,1,0
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,NaN,1.0,Electronics,...,Components & Parts,missing,missing,0,0,0,1,1,0,0
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,NaN,2.0,Women,...,Blouse,missing,missing,0,0,0,1,1,0,0
3,missing,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,NaN,3.0,Home,...,Home Décor Accents,missing,missing,0,0,0,1,1,1,0
4,missing,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,NaN,4.0,Women,...,Necklaces,missing,missing,0,0,0,1,1,1,0


In [9]:
NAME_MIN_DF = 10 ## lower bound for word count
MAX_FEAT_DESCP = 5000

print("Encodings")

count = CountVectorizer(min_df=NAME_MIN_DF)
X_name = count.fit_transform(merged["name"])
X_name

print("Category Encoders")
count_category = CountVectorizer()
X_category = count_category.fit_transform(merged["category_name"])

print("Descp encoders")
count_descp = TfidfVectorizer(max_features = MAX_FEAT_DESCP, 
                              ngram_range = (1,3),
                              stop_words = "english")
X_descp = count_descp.fit_transform(merged["item_description"])

print("Brand encoders")
vect_brand = LabelBinarizer(sparse_output=True)
X_brand = vect_brand.fit_transform(merged["brand_name"])

print("Dummy Encoders")
merged['item_condition_id'] = merged['item_condition_id'].astype('category')
merged['shipping'] = merged['shipping'].astype('category')
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(merged[["item_condition_id", "shipping"]], sparse = True).values)



Encodings
Category Encoders
Descp encoders
Brand encoders
Dummy Encoders


In [10]:
merged.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,general_cat,...,subcat_2,subcat_3,subcat_4,mis_general_cat,mis_subcat_1,mis_subcat_2,mis_subcat_3,mis_subcat_4,mis_brand_name,mis_item_description
0,missing,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,NaN,0.0,Men,...,T-shirts,missing,missing,0,0,0,1,1,1,0
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,NaN,1.0,Electronics,...,Components & Parts,missing,missing,0,0,0,1,1,0,0
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,NaN,2.0,Women,...,Blouse,missing,missing,0,0,0,1,1,0,0
3,missing,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,NaN,3.0,Home,...,Home Décor Accents,missing,missing,0,0,0,1,1,1,0
4,missing,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,NaN,4.0,Women,...,Necklaces,missing,missing,0,0,0,1,1,1,0


In [15]:
X = scipy.sparse.hstack((X_dummies, 
                         X_descp,
                         X_brand,
                         X_category,
                         X_name)).tocsr()
del X_dummies, X_descp, X_brand,X_category,X_name
gc.collect()

NameError: name 'X_dummies' is not defined

In [29]:
##  Save model data
##

scipy.sparse.save_npz("model/x.npz",X)
merged.to_csv("model/merged.csv",encoding='utf-8')

In [2]:
## Load data
##

X = scipy.sparse.load_npz("model/x.npz")
merged = pd.read_csv("model/merged.csv")
nrow_train = 1482535

In [3]:
X.shape

(2175894, 29252)

In [3]:
y_merged_train = merged['price'][:nrow_train]
y_merged_train = np.log1p(y_merged_train)
X_merged_train = X[:nrow_train]
X_test = X[nrow_train:]

X_train, X_valid, y_train, y_valid = train_test_split(X_merged_train, y_merged_train, test_size=0.2)
k_fold = KFold(n_splits=5,random_state=101)


print (X_train.shape, len(y_train))
print (X_valid.shape, len(y_valid))
print (X_test.shape)


(1186028, 29252) 1186028
(296507, 29252) 296507
(693359, 29252)


array([ 1. ,  0.9,  0.8,  0.7,  0.6,  0.5,  0.4,  0.3,  0.2,  0.1])

In [8]:
## https://www.kaggle.com/apapiu/ridge-script
##

scorer_func = make_scorer(mean_squared_error,greater_is_better=False)

for val_alpha in np.arange(1,0,-0.1):
    mm = Ridge(alpha = val_alpha, solver = "lsqr", normalize = False, fit_intercept=False)
    scores = cross_val_score(mm, X_train, y_train, 
                             cv=k_fold, n_jobs=-1,scoring = scorer_func)
    scores = np.sqrt(-scores)
    print (val_alpha,scores,np.mean(scores))

## test on validation
mm.fit(X_train, y_train)
y_pred = mm.predict(X_valid)


print ("RMSLE: ",rmsle2(np.expm1(y_valid),np.expm1(y_pred)))
print ("R2: ",r2_score(y_valid,y_pred))

1.0 [ 0.48383854  0.48334505  0.48304061  0.48288834  0.48312447]
0.9 [ 0.4838456   0.48335081  0.48304728  0.4828963   0.48313102]
0.8 [ 0.48385276  0.48335668  0.48305406  0.48290436  0.48313768]


KeyboardInterrupt: 

In [ ]:
## Predicting csv data
##

sub = pd.DataFrame()
y_pred = mm.predict(X_test)
y_pred[y_pred<1.386]=1.386

sub['test_id'] = merged['test_id'][nrow_train:]
sub['test_id'] = sub['test_id'].astype(int)
sub['price'] = y_pred

sub.head()
sub.to_csv("sub_mm.csv", index=False)

In [12]:
params = {
        'learning_rate': 0.65,
        'application': 'regression',
        'max_depth': 4,
        'num_leaves': 42,
        'verbosity': -1,
        'metric': 'RMSE',
        'data_random_seed': 1,
        'bagging_fraction': 0.71,
        'bagging_freq': 5,
        'feature_fraction': 0.67,
        'nthread': 4,
        'min_data_in_leaf': 120,
        'max_bin': 40
    }

d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid)


watchlist2 = [d_valid]    
model = lgb.train(params, train_set=d_train, num_boost_round=8000, valid_sets=watchlist2, \
    early_stopping_rounds=250, verbose_eval=True)     

y_pred = model.predict(X_valid)

y_pred[y_pred<0] = 0
print ("RMSLE: ",rmsle(y_valid,y_pred))
print ("RMSE: ",mean_squared_error(y_valid,y_pred))
print ("R2: ",r2_score(y_valid,y_pred))


[1]	valid_0's rmse: 0.695642
Training until validation scores don't improve for 250 rounds.
[2]	valid_0's rmse: 0.676043
[3]	valid_0's rmse: 0.664923
[4]	valid_0's rmse: 0.6534
[5]	valid_0's rmse: 0.64046
[6]	valid_0's rmse: 0.632561
[7]	valid_0's rmse: 0.626064
[8]	valid_0's rmse: 0.621221
[9]	valid_0's rmse: 0.616337
[10]	valid_0's rmse: 0.612735
[11]	valid_0's rmse: 0.608997
[12]	valid_0's rmse: 0.606567
[13]	valid_0's rmse: 0.603846
[14]	valid_0's rmse: 0.601374
[15]	valid_0's rmse: 0.598811
[16]	valid_0's rmse: 0.596648
[17]	valid_0's rmse: 0.594832
[18]	valid_0's rmse: 0.592867
[19]	valid_0's rmse: 0.590855
[20]	valid_0's rmse: 0.588972
[21]	valid_0's rmse: 0.587296
[22]	valid_0's rmse: 0.585878
[23]	valid_0's rmse: 0.58365
[24]	valid_0's rmse: 0.582689
[25]	valid_0's rmse: 0.581067
[26]	valid_0's rmse: 0.579788
[27]	valid_0's rmse: 0.578498
[28]	valid_0's rmse: 0.577471
[29]	valid_0's rmse: 0.576024
[30]	valid_0's rmse: 0.574788
[31]	valid_0's rmse: 0.573629
[32]	valid_0's rmse:

[269]	valid_0's rmse: 0.507133
[270]	valid_0's rmse: 0.506994
[271]	valid_0's rmse: 0.506812
[272]	valid_0's rmse: 0.506741
[273]	valid_0's rmse: 0.506667
[274]	valid_0's rmse: 0.506563
[275]	valid_0's rmse: 0.506516
[276]	valid_0's rmse: 0.506391
[277]	valid_0's rmse: 0.506306
[278]	valid_0's rmse: 0.506247
[279]	valid_0's rmse: 0.506137
[280]	valid_0's rmse: 0.506038
[281]	valid_0's rmse: 0.505964
[282]	valid_0's rmse: 0.505901
[283]	valid_0's rmse: 0.505808
[284]	valid_0's rmse: 0.505743
[285]	valid_0's rmse: 0.505676
[286]	valid_0's rmse: 0.505635
[287]	valid_0's rmse: 0.505577
[288]	valid_0's rmse: 0.505481
[289]	valid_0's rmse: 0.505446
[290]	valid_0's rmse: 0.50538
[291]	valid_0's rmse: 0.505313
[292]	valid_0's rmse: 0.505256
[293]	valid_0's rmse: 0.505213
[294]	valid_0's rmse: 0.505165
[295]	valid_0's rmse: 0.505142
[296]	valid_0's rmse: 0.505059
[297]	valid_0's rmse: 0.504983
[298]	valid_0's rmse: 0.504927
[299]	valid_0's rmse: 0.504861
[300]	valid_0's rmse: 0.504733
[301]	val

[535]	valid_0's rmse: 0.491152
[536]	valid_0's rmse: 0.491104
[537]	valid_0's rmse: 0.491089
[538]	valid_0's rmse: 0.491048
[539]	valid_0's rmse: 0.491023
[540]	valid_0's rmse: 0.490999
[541]	valid_0's rmse: 0.490964
[542]	valid_0's rmse: 0.490949
[543]	valid_0's rmse: 0.490852
[544]	valid_0's rmse: 0.49082
[545]	valid_0's rmse: 0.490792
[546]	valid_0's rmse: 0.490748
[547]	valid_0's rmse: 0.490727
[548]	valid_0's rmse: 0.490686
[549]	valid_0's rmse: 0.490666
[550]	valid_0's rmse: 0.490631
[551]	valid_0's rmse: 0.49063
[552]	valid_0's rmse: 0.490627
[553]	valid_0's rmse: 0.490607
[554]	valid_0's rmse: 0.490594
[555]	valid_0's rmse: 0.490555
[556]	valid_0's rmse: 0.490542
[557]	valid_0's rmse: 0.490477
[558]	valid_0's rmse: 0.490458
[559]	valid_0's rmse: 0.490441
[560]	valid_0's rmse: 0.490421
[561]	valid_0's rmse: 0.490404
[562]	valid_0's rmse: 0.490392
[563]	valid_0's rmse: 0.490367
[564]	valid_0's rmse: 0.49033
[565]	valid_0's rmse: 0.490282
[566]	valid_0's rmse: 0.490228
[567]	valid

[801]	valid_0's rmse: 0.484731
[802]	valid_0's rmse: 0.484718
[803]	valid_0's rmse: 0.484707
[804]	valid_0's rmse: 0.484682
[805]	valid_0's rmse: 0.484672
[806]	valid_0's rmse: 0.48467
[807]	valid_0's rmse: 0.484659
[808]	valid_0's rmse: 0.484659
[809]	valid_0's rmse: 0.484621
[810]	valid_0's rmse: 0.484612
[811]	valid_0's rmse: 0.484595
[812]	valid_0's rmse: 0.48458
[813]	valid_0's rmse: 0.484568
[814]	valid_0's rmse: 0.484569
[815]	valid_0's rmse: 0.484569
[816]	valid_0's rmse: 0.484553
[817]	valid_0's rmse: 0.484543
[818]	valid_0's rmse: 0.4845
[819]	valid_0's rmse: 0.484487
[820]	valid_0's rmse: 0.484448
[821]	valid_0's rmse: 0.484405
[822]	valid_0's rmse: 0.484396
[823]	valid_0's rmse: 0.484385
[824]	valid_0's rmse: 0.484389
[825]	valid_0's rmse: 0.484374
[826]	valid_0's rmse: 0.484367
[827]	valid_0's rmse: 0.484349
[828]	valid_0's rmse: 0.484365
[829]	valid_0's rmse: 0.484363
[830]	valid_0's rmse: 0.484342
[831]	valid_0's rmse: 0.484334
[832]	valid_0's rmse: 0.484314
[833]	valid_

[1065]	valid_0's rmse: 0.480118
[1066]	valid_0's rmse: 0.480037
[1067]	valid_0's rmse: 0.480033
[1068]	valid_0's rmse: 0.479989
[1069]	valid_0's rmse: 0.479984
[1070]	valid_0's rmse: 0.47998
[1071]	valid_0's rmse: 0.479963
[1072]	valid_0's rmse: 0.47994
[1073]	valid_0's rmse: 0.479914
[1074]	valid_0's rmse: 0.479903
[1075]	valid_0's rmse: 0.47989
[1076]	valid_0's rmse: 0.479882
[1077]	valid_0's rmse: 0.47987
[1078]	valid_0's rmse: 0.479863
[1079]	valid_0's rmse: 0.479857
[1080]	valid_0's rmse: 0.479854
[1081]	valid_0's rmse: 0.479863
[1082]	valid_0's rmse: 0.479847
[1083]	valid_0's rmse: 0.479833
[1084]	valid_0's rmse: 0.479824
[1085]	valid_0's rmse: 0.479803
[1086]	valid_0's rmse: 0.479793
[1087]	valid_0's rmse: 0.479774
[1088]	valid_0's rmse: 0.479757
[1089]	valid_0's rmse: 0.479756
[1090]	valid_0's rmse: 0.479749
[1091]	valid_0's rmse: 0.479739
[1092]	valid_0's rmse: 0.47974
[1093]	valid_0's rmse: 0.479738
[1094]	valid_0's rmse: 0.479729
[1095]	valid_0's rmse: 0.479728
[1096]	valid_

...

[4161]	valid_0's rmse: 0.471228
[4162]	valid_0's rmse: 0.471228
[4163]	valid_0's rmse: 0.47122
[4164]	valid_0's rmse: 0.471229
[4165]	valid_0's rmse: 0.471234
[4166]	valid_0's rmse: 0.471242
[4167]	valid_0's rmse: 0.471246
[4168]	valid_0's rmse: 0.471244
[4169]	valid_0's rmse: 0.471239
[4170]	valid_0's rmse: 0.471238
[4171]	valid_0's rmse: 0.471241
[4172]	valid_0's rmse: 0.471235
[4173]	valid_0's rmse: 0.471233
[4174]	valid_0's rmse: 0.471223
[4175]	valid_0's rmse: 0.471233
[4176]	valid_0's rmse: 0.471235
[4177]	valid_0's rmse: 0.471232
[4178]	valid_0's rmse: 0.471236
[4179]	valid_0's rmse: 0.471226
[4180]	valid_0's rmse: 0.471222
[4181]	valid_0's rmse: 0.471215
[4182]	valid_0's rmse: 0.471213
[4183]	valid_0's rmse: 0.471219
[4184]	valid_0's rmse: 0.471228
[4185]	valid_0's rmse: 0.471232
[4186]	valid_0's rmse: 0.471233
[4187]	valid_0's rmse: 0.471235
[4188]	valid_0's rmse: 0.471232
[4189]	valid_0's rmse: 0.471229
[4190]	valid_0's rmse: 0.471236
[4191]	valid_0's rmse: 0.471232
[4192]	va

[4419]	valid_0's rmse: 0.4712
[4420]	valid_0's rmse: 0.471202
[4421]	valid_0's rmse: 0.471204
[4422]	valid_0's rmse: 0.471206
[4423]	valid_0's rmse: 0.471205
[4424]	valid_0's rmse: 0.471201
[4425]	valid_0's rmse: 0.4712
[4426]	valid_0's rmse: 0.471191
[4427]	valid_0's rmse: 0.471192
[4428]	valid_0's rmse: 0.471193
[4429]	valid_0's rmse: 0.471186
[4430]	valid_0's rmse: 0.471184
[4431]	valid_0's rmse: 0.471181
[4432]	valid_0's rmse: 0.471174
[4433]	valid_0's rmse: 0.471165
[4434]	valid_0's rmse: 0.471159
[4435]	valid_0's rmse: 0.471151
[4436]	valid_0's rmse: 0.471158
[4437]	valid_0's rmse: 0.471159
[4438]	valid_0's rmse: 0.471164
[4439]	valid_0's rmse: 0.471166
[4440]	valid_0's rmse: 0.47116
[4441]	valid_0's rmse: 0.471167
[4442]	valid_0's rmse: 0.471164
[4443]	valid_0's rmse: 0.47116
[4444]	valid_0's rmse: 0.471163
[4445]	valid_0's rmse: 0.471162
[4446]	valid_0's rmse: 0.471167
[4447]	valid_0's rmse: 0.471166
[4448]	valid_0's rmse: 0.471171
[4449]	valid_0's rmse: 0.471176
[4450]	valid_0

[4677]	valid_0's rmse: 0.471125
[4678]	valid_0's rmse: 0.471124
[4679]	valid_0's rmse: 0.471124
[4680]	valid_0's rmse: 0.471126
[4681]	valid_0's rmse: 0.471128
[4682]	valid_0's rmse: 0.471118
[4683]	valid_0's rmse: 0.471119
[4684]	valid_0's rmse: 0.471109
[4685]	valid_0's rmse: 0.471116
[4686]	valid_0's rmse: 0.471118
[4687]	valid_0's rmse: 0.47112
[4688]	valid_0's rmse: 0.471134
[4689]	valid_0's rmse: 0.471132
[4690]	valid_0's rmse: 0.471136
[4691]	valid_0's rmse: 0.471138
[4692]	valid_0's rmse: 0.47114
[4693]	valid_0's rmse: 0.471137
[4694]	valid_0's rmse: 0.47114
[4695]	valid_0's rmse: 0.47112
[4696]	valid_0's rmse: 0.471118
[4697]	valid_0's rmse: 0.471134
[4698]	valid_0's rmse: 0.471131
[4699]	valid_0's rmse: 0.471132
[4700]	valid_0's rmse: 0.47113
[4701]	valid_0's rmse: 0.471115
[4702]	valid_0's rmse: 0.471114
[4703]	valid_0's rmse: 0.471102
[4704]	valid_0's rmse: 0.471102
[4705]	valid_0's rmse: 0.471086
[4706]	valid_0's rmse: 0.471081
[4707]	valid_0's rmse: 0.471087
[4708]	valid_

[4935]	valid_0's rmse: 0.470995
[4936]	valid_0's rmse: 0.470997
[4937]	valid_0's rmse: 0.471001
[4938]	valid_0's rmse: 0.471002
[4939]	valid_0's rmse: 0.471003
[4940]	valid_0's rmse: 0.470999
[4941]	valid_0's rmse: 0.47101
[4942]	valid_0's rmse: 0.471016
[4943]	valid_0's rmse: 0.471023
[4944]	valid_0's rmse: 0.471019
[4945]	valid_0's rmse: 0.471013
[4946]	valid_0's rmse: 0.471011
[4947]	valid_0's rmse: 0.471016
[4948]	valid_0's rmse: 0.471012


KeyboardInterrupt: 